# Re-usable code to display financial metrics in our final app

This notebook serves as a point of reference & experimentation for various functions we can re-use for our data science project

In [69]:
# data handling
import pandas as pd
import numpy as np
# finance API
# Limit: 2000 requests per hour per IP, max 48'000 per day.
import yfinance as yf
# time & date packages
import time, datetime
from dateutil import parser
# API requests
import requests
# file handling
import io
# graphing
import plotly.graph_objects as go

# Retrieve financial information from a list of tickers



In [67]:
# note: the function expects dates passed in "datetime" format, 
# and array of string tickers without $, such as "AAPL", not "$AAPL"
# example: ['AAPL', 'TSLA', 'NVDA']
def fetch_financial_data(tickers, start_date, end_date):
    
    # protective code in case a single string ticker was passed instead of an array:
    if type(tickers) == str:
        tickers = [tickers]

    
    # protective code in case dates were passed as strings instead of datetime:
    if type(start_date) == str:
        start_date = parser.parse(start_date)
    if type(end_date) == str:
        end_date = parser.parse(end_date)
    
    # Dataframe to store the stocks desired
    stocks = pd.DataFrame()
    
    for ticker in tickers:
        print(f"Fetching financial data about {ticker}...")
        
        # removing the $ in front of the ticker name if there is one
        if ticker == '$':
            ticker = ticker[1:]
        stock = None # bug if you remove this line for some reason?
        stock = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        # If financial info was not found:
        if stock.empty:
            print(f"{stock} was not found on Yahoo Finance. Discarding")
        else:
            stock['ticker_name'] = ticker
            stocks = stocks.append(stock)
        
    # ---
    return stocks

In [68]:
# Example. It works!
fetch_financial_data(['TSLA', 'AAPL'], '2020-6-15', '2021/6/14')

Fetching financial data about TSLA...
Fetching financial data about AAPL...


,Open,High,Low,Close,Adj Close,Volume,ticker_name
Date,,,,,,,
2020-06-15,183.557999,199.768005,181.699997,198.179993,198.179993,78486000,TSLA
2020-06-16,202.369995,202.576004,192.477997,196.425995,196.425995,70255500,TSLA
2020-06-17,197.542007,201.000000,196.514008,198.358002,198.358002,49454000,TSLA
2020-06-18,200.600006,203.839996,198.893997,200.792007,200.792007,48759500,TSLA
2020-06-19,202.556000,203.194000,198.268005,200.179993,200.179993,43398500,TSLA
...,...,...,...,...,...,...,...
2021-06-07,126.169998,126.320000,124.830002,125.900002,125.368721,71057600,AAPL
2021-06-08,126.599998,128.460007,126.209999,126.739998,126.205177,74403800,AAPL
2021-06-09,127.209999,127.750000,126.519997,127.129997,126.593529,56877900,AAPL


In [25]:
# the function is flexible in formatting!
# All of the following formats are acceptable and will be handled
fetch_financial_data('AAPL', '2020-6-15', '2021/6/14')
fetch_financial_data('$AAPL', '2020-6-15', '2021/6/14')
fetch_financial_data(['$AAPL'], '2020-6-15', '2021/6/14')
fetch_financial_data(['$AAPL', 'TSLA', 'DATAX???'], '2020-6-15', '2021/6/14')
fetch_financial_data(['$AAPL'], datetime.dateime(2020,6,15), '2021/6/14')

['AAPL'] 2020-06-15 00:00:00 2021-06-14 00:00:00 364 days, 0:00:00


# Graphing Historical Data

In [89]:
# without using previous function
# DOCUMENTATION TO CUSTOMIZE PLOT: https://plotly.com/python/figure-labels/

# ALLOWED PERIOD: xm, xh, xd, xwk, xmo, xy, max
def plot_single_stock(ticker, period):
    if ticker[0] == "$":
        ticker = ticker[1:]
    print(ticker)
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    plot = go.Figure(data=go.Scatter(x=history.index, y=history['Close'], 
                                     mode='lines'))
    plot.update_layout(title=f"Stock: {ticker}, Period: {period}")
    plot.show()

In [90]:
plot_single_stock('TSLA', '1y')

TSLA


In [91]:
plot_single_stock(ticker='$AAPL', period='max')

AAPL


In [92]:
plot_single_stock(ticker='NVDA', period='5d')

NVDA
